## Scan for Yaml files:

In [72]:
from pathlib import Path

root = '.'
path = './ressources_from_aides_jeunes/aides/'

current_path = f'{root}/{path}'
benefits_folder = Path(current_path)

def extract_benefits_paths(benefits_folder):
    isYAMLfile = lambda path : str(path).endswith('.yml') or str(path).endswith('.yaml')
    liste_fichiers = [str(benefit) for benefit in benefits_folder.iterdir() if isYAMLfile(benefit)]
    return liste_fichiers

benefits_files_paths = extract_benefits_paths(benefits_folder)
benefits_files_paths

['ressources_from_aides_jeunes/aides/haut-de-france-aide-transports-salaries.yml',
 'ressources_from_aides_jeunes/aides/region-auvergne-rhone-alpes-billet-illico-autocars.yml',
 'ressources_from_aides_jeunes/aides/hauts-de-france-mon-abo-etudiant-ter.yml',
 'ressources_from_aides_jeunes/aides/region_bretagne-b-mouve-–-mobilité-des-formations-sanitaires-et-sociales.yml',
 'ressources_from_aides_jeunes/aides/region_nouvelle_aquitaine-stages-à-létranger-–-infra-bac.yml',
 'ressources_from_aides_jeunes/aides/caf-territoire-de-belfort-aide-bafa-approfondissement.yml',
 'ressources_from_aides_jeunes/aides/region-nouvelle-aquitaine-sas-jeunes-orientation-active-vers-emploi.yml',
 'ressources_from_aides_jeunes/aides/region-auvergne-rhone-alpes-abonnement-illico.yml',
 'ressources_from_aides_jeunes/aides/caf-haute-loire-aide-bafa-approfondissement.yml',
 'ressources_from_aides_jeunes/aides/region_la_reunion-bourse-régionale-sanitaire-et-sociale.yml',
 'ressources_from_aides_jeunes/aides/caf-sar

## Open and parse files: make a list of Dict

In [ ]:
import yaml

def extract_benefit_file_content(aide_path):
    benefit:dict = yaml.safe_load(open(aide_path))
    benefit['file_path'] = aide_path
    return benefit

def extract_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [extract_benefit_file_content(path) for path in benefits_files_paths]
    return benefits

#### Making benefits objects (bad idea, dont use `benefits_objects`)

In [ ]:
from benefit import Benefit
def parse_benefits_from_yamls(path):
    benefits_files_paths = extract_benefits_paths(path)
    benefits = [Benefit(extract_benefit_file_content(path)) for path in benefits_files_paths]
    return benefits

benefits_objects = parse_benefits_from_yamls(benefits_folder)    

In [ ]:
benefits = extract_benefits_from_yamls(benefits_folder)
len(benefits)

##  Make a list of [profils] [type] in all aides given

In [ ]:

from functools import reduce
from benefit import BenefitProfile

def accumulate_profils(profils: list, benefit: BenefitProfile):
        return profils + benefit.get_profils_types()

profiles_names = reduce(accumulate_profils, benefits_objects, [])


# Make a list with all conditons:
	- conditions generales
	- conditions in [profils][type]

In [ ]:
from functools import reduce

from benefit import BenefitCondition

def accumulate_conditions(conditions: list, benefit: BenefitCondition):
    return conditions + benefit.get_all_conditions_types()

conditions_names = reduce(accumulate_conditions, benefits_objects, [])
conditions_names

# Count frequence of profils and conditions appearances

In [ ]:
def count_list_item_frequence(list : list):
    elements_counter = {}
    for element in list:
        if element in elements_counter:
            elements_counter[element] +=1
        else:
            elements_counter[element] = 1
    return (sorted(elements_counter.items(), key=lambda x:x[1]))

profil_names_frequence = count_list_item_frequence(profiles_names)
condition_frequence = count_list_item_frequence(conditions_names)

In [ ]:
condition_frequence

In [ ]:
profil_names_frequence

## Reference benefits with no profils:

In [ ]:
def contains_profils(benefit :dict):
    return 'profils' in benefit and benefit['profils'] != []

def extract_benefit_without_profile(benefit: Benefit):
    if benefit.profils == []:
        return [benefit.file_path]
    else:
        return []

def accumulate_benefit_without_profile(list:list, benefit):
    
    return list + extract_benefit_without_profile(benefit)
    
    
benefits_without_profils = reduce(accumulate_benefit_without_profile, benefits_objects, [])
benefits_without_profils

## Benefits using only age as condition:

In [ ]:
def is_only_condition_age(benefit: Benefit):
    conditions = benefit.get_all_conditions_types()
    return all(condition == "age" for condition in conditions) and conditions != []

def extract_benefit_with_only_condition(benefit : Benefit):
    if is_only_condition_age(benefit):
        return [benefit.file_path]
    else:
        return []

def accumulate_benefit_with_only_condition(list:list, benefit: Benefit):
    
    return list + extract_benefit_with_only_condition(benefit)

benefits_with_only_age_condition = reduce(accumulate_benefit_with_only_condition, benefits_objects, [])
benefits_with_only_age_condition

## Getting benefits Conditions (tools)

In [ ]:
def get_benefit_profil_conditions(benefit):
    conditions = []
    if 'profile' in benefit and 'conditions' in benefit['profile']:
        conditions= [condition for profile in benefit['profils'] for condition in profile['conditions']]
    return conditions

def get_benefit_general_conditions(benefit):
    return [condition for condition in benefit['conditions_generales']]

def get_all_benefit_conditions(benefit):
    return get_benefit_general_conditions(benefit) + get_benefit_profil_conditions(benefit)

## Get Benefits with 1 conditions

In [ ]:
def get_benefit_with_1_conditions(benefits):
    return [benefit for benefit in benefits if len(get_all_benefit_conditions(benefit)) == 1]

benefits_with_1_conditions =  get_benefit_with_1_conditions(benefits)
len(benefits_with_1_conditions)

#### Benefit conditions tools

In [ ]:
def get_condition_type(condition):
    return condition['type']

def contains_age_condition(benefit):
    conditions = get_all_benefit_conditions(benefit)
    return 'age' in [get_condition_type(condition) for condition in conditions]

def contains_departments_condition(benefit):
    conditions = get_all_benefit_conditions(benefit)
    return 'departements' in [get_condition_type(condition) for condition in conditions]

def contains_regions_condition(benefit):
    conditions = get_all_benefit_conditions(benefit)
    return 'regions' in [get_condition_type(condition) for condition in conditions]

def unwanted_conditions(benefit):
    return contains_age_condition(benefit) or contains_departments_condition(benefit) or contains_regions_condition(benefit)



#### Get Benefits with 1 conditions which is not age

In [ ]:
benefits_with_1_conditions_not_age = [benefit for benefit in benefits_with_1_conditions if not contains_age_condition(benefit)]
len(benefits_with_1_conditions_not_age)

In [ ]:
[(benefit['file_path'].split('/')[-1]) for benefit in benefits_with_1_conditions_not_age]

### Get Benefits with 1 conditions which is not age nor region nor departements

In [ ]:
benefits_with_1_conditions_not_age_regions_departments = [benefit for benefit in benefits_with_1_conditions if not unwanted_conditions(benefit)]
len(benefits_with_1_conditions_not_age_regions_departments)